In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json

In [3]:
bus = 'yelp_data/health_business01.csv'

In [4]:
review_dataset = 'yelp_data/yelp_academic_dataset_review.json'

# Load `yelp_academic_dataset_review.json`
#### size :

- 4.6 gigs

#### Hardware

````
w10
    description: Computer
    width: 64 bits
  *-core
       description: Motherboard
       physical id: 0
     *-memory
          description: System memory
          physical id: 0
          size: 47GiB
     *-cpu:0
          product: Intel(R) Xeon(R) CPU           X5667  @ 3.07GHz
          vendor: Intel Corp.
          physical id: 1
          bus info: cpu@0
          capacity: 3060MHz
          width: 64 bits
          capabilities: fpu fpu_exception wp vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp x86-64 pni pclmulqdq est tm2 ssse3 cx16 xtpr pdcm pcid sse4_1 sse4_2 popcnt aes hypervisor cpufreq
     *-cpu:1
          product: Intel(R) Xeon(R) CPU           X5667  @ 3.07GHz
          vendor: Intel Corp.
          physical id: 2
          bus info: cpu@1
          capacity: 3060MHz
          width: 64 bits
          capabilities: fpu fpu_exception wp vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp x86-64 pni pclmulqdq est tm2 ssse3 cx16 xtpr pdcm pcid sse4_1 sse4_2 popcnt aes hypervisor cpufreq
````

#### Time
```
CPU times: user 1min 52s, sys: 1min 9s, total: 3min 2s
Wall time: 3min
```

In [5]:
%%time
with open(review_dataset, encoding="utf-8") as f:
    data = pd.DataFrame(json.loads(line) for line in f)

CPU times: user 1min 51s, sys: 1min 13s, total: 3min 5s
Wall time: 3min 3s


In [6]:
list(data.columns)

['business_id',
 'cool',
 'date',
 'funny',
 'review_id',
 'stars',
 'text',
 'useful',
 'user_id']

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5996996 entries, 0 to 5996995
Data columns (total 9 columns):
business_id    object
cool           int64
date           object
funny          int64
review_id      object
stars          int64
text           object
useful         int64
user_id        object
dtypes: int64(4), object(5)
memory usage: 411.8+ MB


In [8]:
data.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,0,msQe1u7Z_XuqjGoqhB0J5g
1,pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,dDl8zu1vWPdKGihJrwQbpw,5,I love this place! My fiance And I go here atl...,0,msQe1u7Z_XuqjGoqhB0J5g
2,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,1,LZp4UX5zK3e-c5ZGSeo3kA,1,Terrible. Dry corn bread. Rib tips were all fa...,3,msQe1u7Z_XuqjGoqhB0J5g
3,elqbBhBfElMNSrjFqW3now,0,2011-02-25,0,Er4NBWCmCD4nM8_p1GRdow,2,Back in 2005-2007 this place was my FAVORITE t...,2,msQe1u7Z_XuqjGoqhB0J5g
4,Ums3gaP2qM3W1XcA5r6SsQ,0,2014-09-05,0,jsDu6QEJHbwP2Blom1PLCA,5,Delicious healthy food. The steak is amazing. ...,0,msQe1u7Z_XuqjGoqhB0J5g


# load Our new Healthcare business dataset

- merge with our data dataset
    - on `business_id`

In [9]:
df = pd.read_csv(bus)
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [10]:
df.head()

,business_id,categories,name,state,categories_list
0,y-4xTZNKVm8mAZpiXMS5ZA,"urologists, doctors, health & medical","Lauren Byrne, MD",AZ,"['urologists', ' doctors', ' health & medical']"
1,STcSSjPOZ0FWtjvjqw99Dg,"doctors, orthopedists, health & medical",Black Mountain Orthopaedics,NV,"['doctors', ' orthopedists', ' health & medical']"
2,WnRtn80rdsWOa2nWZ6G1iA,"obstetricians & gynecologists, health & medica...",Central Phoenix Women's Health Care,AZ,"['obstetricians & gynecologists', ' health & m..."
3,EjypAFLWk7k5gk4ebPJANQ,"psychologists, counseling & mental health, nat...",West Valley Naturopathic Center,AZ,"['psychologists', ' counseling & mental health..."
4,yOJrDZU5zkxJoZOJTZXLvQ,"neurologist, spine surgeons, cosmetic surgeons...","Michael Seiff, MD",NV,"['neurologist', ' spine surgeons', ' cosmetic ..."


In [11]:
data.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,0,msQe1u7Z_XuqjGoqhB0J5g
1,pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,dDl8zu1vWPdKGihJrwQbpw,5,I love this place! My fiance And I go here atl...,0,msQe1u7Z_XuqjGoqhB0J5g
2,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,1,LZp4UX5zK3e-c5ZGSeo3kA,1,Terrible. Dry corn bread. Rib tips were all fa...,3,msQe1u7Z_XuqjGoqhB0J5g
3,elqbBhBfElMNSrjFqW3now,0,2011-02-25,0,Er4NBWCmCD4nM8_p1GRdow,2,Back in 2005-2007 this place was my FAVORITE t...,2,msQe1u7Z_XuqjGoqhB0J5g
4,Ums3gaP2qM3W1XcA5r6SsQ,0,2014-09-05,0,jsDu6QEJHbwP2Blom1PLCA,5,Delicious healthy food. The steak is amazing. ...,0,msQe1u7Z_XuqjGoqhB0J5g


In [12]:
%%time
health = pd.merge(df,data, on='business_id')

CPU times: user 8.8 s, sys: 2.88 s, total: 11.7 s
Wall time: 10.1 s


In [13]:
health.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64006 entries, 0 to 64005
Data columns (total 13 columns):
business_id        64006 non-null object
categories         64006 non-null object
name               64006 non-null object
state              64006 non-null object
categories_list    64006 non-null object
cool               64006 non-null int64
date               64006 non-null object
funny              64006 non-null int64
review_id          64006 non-null object
stars              64006 non-null int64
text               64006 non-null object
useful             64006 non-null int64
user_id            64006 non-null object
dtypes: int64(4), object(9)
memory usage: 6.8+ MB


In [14]:
health.head()

,business_id,categories,name,state,categories_list,cool,date,funny,review_id,stars,text,useful,user_id
0,y-4xTZNKVm8mAZpiXMS5ZA,"urologists, doctors, health & medical","Lauren Byrne, MD",AZ,"['urologists', ' doctors', ' health & medical']",0,2018-03-03,0,TNNkSmMfshsD3G60jTNjDA,1,Please stay away from this place if you can! I...,2,xv2V2GO5IZYvtw4oW7gQ1w
1,y-4xTZNKVm8mAZpiXMS5ZA,"urologists, doctors, health & medical","Lauren Byrne, MD",AZ,"['urologists', ' doctors', ' health & medical']",0,2015-11-29,0,v-iKdstPdCxJr8zV1ZMdrw,5,My husband has been a patient of Dr. Byrne for...,1,SjvWP7c9toeZoV_q62zhTA
2,y-4xTZNKVm8mAZpiXMS5ZA,"urologists, doctors, health & medical","Lauren Byrne, MD",AZ,"['urologists', ' doctors', ' health & medical']",0,2016-06-03,0,BmNDRCV9_NzQ_KCChyfdEw,4,Dr. Byrne is a great doctor! She has great bed...,2,sZVHm1aLtvyH9trAc2_MgA
3,y-4xTZNKVm8mAZpiXMS5ZA,"urologists, doctors, health & medical","Lauren Byrne, MD",AZ,"['urologists', ' doctors', ' health & medical']",0,2017-03-17,0,t_TKVMxKFYm9Hl-TIO7UUw,3,I'm raising my review as Dr Bryne's has been m...,3,8Y_irXocZdZxLs_qgzpjBw
4,y-4xTZNKVm8mAZpiXMS5ZA,"urologists, doctors, health & medical","Lauren Byrne, MD",AZ,"['urologists', ' doctors', ' health & medical']",0,2016-08-31,0,NWRrpGRgWZBBj3lvCZGVKA,1,I wish I could give 0 stars. Worst office I've...,1,hVKPDGpG12z7vpScXaSakw


In [15]:
health.head(1).T

,0
business_id,y-4xTZNKVm8mAZpiXMS5ZA
categories,"urologists, doctors, health & medical"
name,"Lauren Byrne, MD"
state,AZ
categories_list,"['urologists', ' doctors', ' health & medical']"
cool,0
date,2018-03-03
funny,0
review_id,TNNkSmMfshsD3G60jTNjDA
stars,1


## Review

- merged our clean healthcare business dataset with yelps review dataset
- total reviews: 64006

### Columns

```
business_id        64006 non-null object
categories         64006 non-null object
name               64006 non-null object
state              64006 non-null object
categories_list    64006 non-null object
cool               64006 non-null int64
date               64006 non-null object
funny              64006 non-null int64
review_id          64006 non-null object
stars              64006 non-null int64
text               64006 non-null object
useful             64006 non-null int64
user_id            64006 non-null object
```

## Save 

In [16]:
%%time
health.to_csv('yelp_data/health_raw00.csv')

CPU times: user 2.17 s, sys: 78.1 ms, total: 2.25 s
Wall time: 2.25 s
